In [ ]:
! pip install kaggle

In [ ]:
! pip install tensorflow.keras.layers.CuDNNLSTM

ERROR: Could not find a version that satisfies the requirement tensorflow.keras.layers.CuDNNLSTM (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.layers.CuDNNLSTM


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d rotchymoricette/25k-dogsvcats

 97% 526M/545M [00:02<00:00, 185MB/s]
100% 545M/545M [00:03<00:00, 188MB/s]


In [ ]:
! unzip 25k-dogsvcats.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tensorFlow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import pickle
from tensorflow.keras.callbacks import TensorBoard
import time

DIRECTORY = r'/content/train'
CATEGORIES = ['cat', 'dog']

IMG_SIZE = 100

data = []

for category in CATEGORIES:
  folder = os.path.join(DIRECTORY, category)  # path to the training data
  label = CATEGORIES.index(category)
  for img in os.listdir(folder):
    try:
      img_arr = cv2.imread(os.path.join(folder, img))
      new_img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
      data.append([new_img_arr, label])
    except Exception as e:
      pass

random.shuffle(data)

X = []
y = []

for features, labels in data:
  X.append(features)
  y.append(labels)

X = np.array(X)
y = np.array(y)

pickle.dump(X, open('X-25k-Dataset.pickle','wb'))
pickle.dump(y, open('y-25k-Dataset.pickle','wb'))

NAME = f'cat-vs.dog-prediction-{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs\\{NAME}\\')

X = pickle.load(open('X-25k-Dataset.pickle','rb'))
y = pickle.load(open('y-25k-Dataset.pickle','rb'))

X = X / 255.0

X.shape

model = Sequential()

model = tensorFlow.expand_dims(model, axis = -1)
model.add(LSTM(128, input_shape = (X.shape[1:]), activation = 'relu', return_sequences = True))
model.add(Dropout(.2))

model.add(LSTM(128), activation = 'relu')
model.add(Dropout(.2))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(.2))

model.add(Dense(10, activation = 'sigmoid'))

#lr is the learning rate
#decay is the rate that the learning rate is going to decrease by (the learning rate takes smaller steps)
myOptimizer = tensorFlow.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = myOptimizer, metrics = ['accuracy'])

model.fit(X, y, epochs = 10, validation_data = (X, y))


ValueError: ignored